In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
'''Importing Data Manipulattion Modules'''
import numpy as np
import pandas as pd

'''Seaborn and Matplotlib Visualization'''
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

'''Importing preprocessing libraries'''
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn import preprocessing

'''Importing keras libraries'''
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ReduceLROnPlateau


Lets load the data

In [ ]:
train=pd.read_csv('../input/Kannada-MNIST/train.csv')
test1=pd.read_csv('../input/Kannada-MNIST/test.csv')
sample_sub=pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')

Lets understand the data


In [ ]:
train.info()

In [ ]:
train.head()

Our training data set has 60000 rows amd 785 columns. First column is label and rest of the 784 colummns are pixel0 to pixel 783.The first column, called label, is the digit that was drawn by the user.The rest of the columns contain the pixel-values of the associated image.

In [ ]:
test1.info()

In [ ]:
test1.head()

Our testing data set has 5000 rows and 785 columns.It does not have label column , but has first column as id column and rest of the columns are pixel0 to pixel783


As per data description , each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total (so each row represent one image in data set). Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

  

Each pixel column in the training set has a name like pixel{x}, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixel{x} is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

Visually, if we omit the "pixel" prefix, the pixels make up the image like this:

000 001 002 003 ----------------------------------------------------------------------------------------------------------------------------------------028
029 030 031 032-----------------------------------------------------------------------------------------------------------------------------------------056


Lets visualize distribution of training set

In [ ]:

y=train.label.value_counts()
sns.barplot(y.index,y)

We have equal counts for all the labels in training set


Lets do data preperation .

Lets seperate features and labels of our training set.And divide our data in taining and vaildation set

In [ ]:
X_train, X_valid, y_train, y_valid =\
    train_test_split(train.drop('label', axis=1).values, train.iloc[:,0].values, 
                     test_size=0.2, random_state=1, stratify=train.iloc[:,0])

test=test1.drop('id',axis=1)
X_test = test.values

print('y_train Shape:', y_train.shape)
print('X_train Shape:', X_train.shape)
print('y_valid Shape:', y_valid.shape)
print('X_valid Shape:', X_valid.shape)
print('X_test Shape: ', X_test.shape)


Next, we need to reshape our dataset inputs ( Xs_train and Xs_test) to the shape that our model expects when we train the model. The first number is the number of images (-1). Then comes the shape of each image (28x28). The last number is 1, which signifies that the images are greyscale.if 3 implies RGB mode

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

print('X_train Shape:', X_train.shape)
print('X_valid Shape:', X_valid.shape)
print('X_test Shape: ', X_test.shape)



Plot the first image in the data set


In [ ]:
plt.imshow(X_train[0][:,:,0])

Neural networks process inputs using small weight values, and inputs with large integer values can disrupt or slow down the learning process. we need to scale pixel values between 0 and 1

In [ ]:
Xs_train = X_train / 255
Xs_valid = X_valid / 255

lets encode our target value.The categorical value assignment can be done using sklearn’s LabelEncoder.Problem with label encoding is that it assumes higher the categorical value, better the category, WHICH WE DONT WANT!lets use Keras inbuild library to_categorical() for one-hot encoding

In [ ]:
y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)

[](http://keras.dhpit.com)


I found these  easy to understand article 

https://www.edureka.co/blog/convolutional-neural-network/

https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/


http://keras.dhpit.com


https://www.dlology.com/blog/one-simple-trick-to-train-keras-model-faster-with-batch-normalization/

https://missinglink.ai/guides/keras/using-keras-flatten-operation-cnn-models-code-examples/

https://keras.io/getting-started/sequential-model-guide/

https://towardsdatascience.com/covolutional-neural-network-cb0883dd6529

http://cs231n.github.io/convolutional-networks/

#define model


#The Sequential model is a linear stack of layers.
model_kannada = Sequential()

In [ ]:
np.random.seed(1)

#The Sequential model is a linear stack of layers.I used the Keras Sequential API, 
#where we add one layer at a time, starting from the input.
from keras.models import Sequential
Kannad_cnn = Sequential()

# adding layers to model
Kannad_cnn.add(Conv2D(32, (3,3), padding='same', input_shape=(28,28,1), activation = 'relu') )
Kannad_cnn.add(MaxPool2D(2,2))
Kannad_cnn.add(BatchNormalization())
#32 is number of neurons(number of convolutional filters to use) in hidden layer
#(3,3)kernel_size

Kannad_cnn.add(Conv2D(64, (3,3), padding='same', activation = 'relu') )
Kannad_cnn.add(MaxPool2D(2,2))
Kannad_cnn.add(BatchNormalization())

Kannad_cnn.add(Flatten())
Kannad_cnn.add(Dense(64, activation='relu'))
Kannad_cnn.add(BatchNormalization())
#In between the convolutional layer and the fully connected layer, there is a ‘Flatten’ layer. Flattening transforms 
#a two-dimensional matrix of features into a vector that can be fed into a fully connected neural network classifier.

Kannad_cnn.add(Dense(10, activation='softmax'))

Kannad_cnn.summary()

#compile Model

In [ ]:
optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08)


Optimizers update the weight parameters to minimize the loss function. Loss function acts as guides to the terrain telling optimizer if it is moving in the right direction to reach the bottom of the valley, the global minimum.There are many optimizers for example adam, SGD, GradientDescent, Adagrad, Adadelta , Adamax,rmsprops optimizer.  


The Adam paper suggests:

Good default settings for the tested machine learning problems are alpha=0.001, beta1=0.9, beta2=0.999 and epsilon=10−8

The TensorFlow documentation suggests some tuning of epsilon:

The default value of 1e-8 for epsilon might not be a good default in general. For example, when training an Inception network on ImageNet a current good choice is 1.0 or 0.1.

We can see that the popular deep learning libraries generally use the default parameters recommended by the paper.

TensorFlow: learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08.
Keras: lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0.

In [ ]:
Kannad_cnn.compile(optimizer=optimizer,loss=['categorical_crossentropy'],metrics=['accuracy'])

A loss function is used to optimize a machine learning algorithm. An accuracy metric is used to measure the algorithm’s performance (accuracy) in an interpretable way. 
Loss is defined as the difference between the predicted value by model and the true value. The most common loss function used in deep neural networks is cross-entropy. 
Accuracy is one of the metrics to measure the performance of  model It’s defined as:
Acccuracy = No of correct predictions/Total no of predictions

In [ ]:
h1 = Kannad_cnn.fit(Xs_train, y_train, batch_size=64, epochs=20, verbose=1, 
                   validation_data = [Xs_valid, y_valid]) 

A batch size is a small part of the dataset.
Iterations is the number of batches of data the algorithm has seen (or simply the number of passes the algorithm has done on the dataset). 
Epochs is the number of times a learning algorithm sees the complete dataset.

By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.
verbose=0 will show you nothing
verbose=1 will show you an animated progress bar
verbose=2 will just mention the number of epoch like this


What Batch size to choose?

Too large a mini-batch size usually leads to a lower accuracy!

There are two notions of speed:

Computational speed
Speed of convergence of an algorithm
Computational speed is simply the speed of performing numerical calculations in hardware. As you said, it is usually higher with a larger mini-batch size. 

the other notion of speed  tells us how quickly our algorithm converges (loss and accuracy we want)

Another "catch" - When using a smaller batch size, calculation of the error has more noise than when we use a larger batch size. however thats not bad as that noise can help the algorithm jump out of a bad local minimum and have more chance of finding either a better local minimum, or hopefully the global minimum.In general, batch size of 32 is a good starting point, and we should also try with 64, 128, and 256. Other values (lower or higher) may be fine for some data sets, but the given range is generally the best to start experimenting with. Though, under 32, it might get too slow because of significantly lower computational speed, because of not exploiting vectorization to the full extent. If you get an "out of memory" error, you should try reducing the mini-batch size anyway.

To conclude,a smaller mini-batch size (not too small) usually leads not only to a smaller number of iterations of a training algorithm, than a large batch size, but also to a higher accuracy overall


In [ ]:
plt.figure(figsize=[12,6])
plt.subplot(1,2,1)
plt.plot(range(1,21), h1.history['accuracy'], label='Training Accuracy')
plt.plot(range(1,21), h1.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(range(1,21), h1.history['loss'], label='Training Loss')
plt.plot(range(1,21), h1.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.legend()

plt.show()

In [ ]:
Y_test = Kannad_cnn.predict_classes(X_test)

In [ ]:
my_submission = pd.DataFrame({'id' :test1. iloc[:,0], 'label':Y_test })

print(my_submission.iloc[:3])

my_submission.to_csv('submission.csv', index=False)